# 10. 인공 신경망 소개
## 10.1 생물학적 뉴런에서 인공 뉴런까지
인공 신경망은 1943년 신경생리학자 워런 매컬러와 수학자 윌터 필츠가 처음 소개했다.
그 후 침체기를 겪었다가 최근에 들어 또 다른 부흥이 시작되었다.

- 신경망을 훈련하기 위한 데이터가 많아짐
- 하드웨어가 크게 발전함
- 훈련 알고리즘이 향상됨
- 기타 등등

### 10.1.1 생물학적 뉴런
### 10.1.2 뉴런을 사용한 논리 연산
### 10.1.3 퍼셉트론
퍼셉트론은 간단한 인공 신경망 구조 중 하나로, 1597년 프랑크 로젠블라트가 제안했다.
입력과 출력이 어떤 숫자이고, 각각의 입력 연결은 가중치와 연관되어 있다.
입력의 가중치 합을 계산한 후 계단함수를 적용하여 결과를 출력한다.

사이킷런에서는 하나의 TLU 네트워크를 구현한 `Perceptron` 클래스를 제공한다. 여기에 붓꽃 데이터를 사용해 보자.

In [4]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:, (2, 3)]
y = (iris.target == 0).astype(np.int8)

per_clf = Perceptron()
per_clf.fit(X, y)

per_clf.predict([[2, 0.5]])

array([0], dtype=int8)